# Prévision des finances sur la base des données

L'objectif est de construire un modèle capable de prédire sur les cinq prochaines années les finances dans un domaine (indicateur) donné.

In [1]:
import pandas as pd 
import re
import spacy as sp


## Lecture et traitement des données

In [2]:
# Lecture du csv
donnees = pd.read_csv("API_TGO_DS2_en_csv_v2_102902/API_TGO_DS2_en_csv_v2_102902.csv", skiprows=5)

# Affichage des données chargées
donnees

,Togo,TGO,"Intentional homicides, male (per 100,000 male)",VC.IHR.PSRC.MA.P5,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69
0,Togo,TGO,Battle-related deaths (number of people),VC.BTL.DETH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.400000e+01,2.600000e+01,NaN,NaN
1,Togo,TGO,Voice and Accountability: Percentile Rank,VA.PER.RNK,NaN,NaN,NaN,NaN,NaN,NaN,...,3.152709e+01,2.906404e+01,2.621359e+01,2.705314e+01,2.801932e+01,2.657005e+01,2.753623e+01,2.647059e+01,NaN,NaN
2,Togo,TGO,Transport services (% of commercial service ex...,TX.VAL.TRAN.ZS.WT,NaN,NaN,NaN,NaN,NaN,NaN,...,4.352915e+01,3.947473e+01,3.751170e+01,3.723193e+01,3.787760e+01,NaN,NaN,NaN,NaN,NaN
3,Togo,TGO,"Computer, communications and other services (%...",TX.VAL.OTHR.ZS.WT,NaN,NaN,NaN,NaN,NaN,NaN,...,3.050142e+01,3.206887e+01,3.272817e+01,3.188613e+01,3.920251e+01,NaN,NaN,NaN,NaN,NaN
4,Togo,TGO,"Merchandise exports by the reporting economy, ...",TX.VAL.MRCH.RS.ZS,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,-1.162746e-14,5.401700e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,Togo,TGO,Adjusted savings: carbon dioxide damage (% of ...,NY.ADJ.DCO2.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,1.359561e+00,1.210120e+00,1.269051e+00,1.334426e+00,1.262783e+00,1.202721e+00,NaN,NaN,NaN,NaN
1504,Togo,TGO,"Services, value added (% of GDP)",NV.SRV.TOTL.ZS,NaN,NaN,NaN,1.728398e+01,2.031254e+01,2.367815e+01,...,5.035119e+01,5.054652e+01,5.053120e+01,5.123722e+01,5.133959e+01,5.141925e+01,5.107844e+01,5.172739e+01,NaN,NaN
1505,Togo,TGO,"Services, value added (current LCU)",NV.SRV.TOTL.CN,NaN,NaN,NaN,8.567367e+09,1.167495e+10,1.534817e+10,...,1.811540e+12,1.874720e+12,1.972940e+12,2.099240e+12,2.186810e+12,2.378670e+12,2.602850e+12,2.877600e+12,NaN,NaN
1506,Togo,TGO,Medium and high-tech manufacturing value added...,NV.MNF.TECH.ZS.UN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
